In [ ]:
# code sketch:
# 1. small subset of songs, get lyrics / info from spotify / genius api
#     turn lyrics coupled w metadata into vector using BERT or something
#.    store in Pinecone? FAISS
# 2. user input > gpt 4 / langchain to get semantic vector rep. of that
#.     compare using cosine sim., closest 10 get suggested


In [1]:
# import all necessary files
from setup import *
from spotify_handler import *
from genius_handler import *
#from langchain_handler import *

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
songNumber =  10000 # number of songs we're gonna sample from randomly
pbar = tqdm(total = songNumber) #progress bar init
query_offset = 998 # number of songs to offset the search by
max_workers = 8 # number of workers to use for the multithreading

build_track_list(songNumber, query_offset, max_workers)
songs = read_from_sql()

  0%|          | 0/1000 [35:11<?, ?it/s]]

  0%|          | 0/10000 [02:29<?, ?it/s]          HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': '%f%', 'limit': 1, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.

  0%|          | 0/10000 [02:31<?, ?it/s]            HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': '%q%', 'limit': 1, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.

  0%|          | 0/10000 [02:38<?, ?it/s]            HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'c%', 'limit': 1, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.

  0%|          | 0/10000 [09:38<?, ?it/s]             HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'x%', 'limit': 1, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.

  0%|          | 0/10000 [12:32<?, ?it/s

In [ ]:
#print(len(trackList[0]))
#print(trackList)
print(songs)

      index                                             Lyrics  \
0         0  51 ContributorsTranslationsPortuguêsEspañolTür...   
1         1  98 ContributorsTranslationsTürkçeEspañolDeutsc...   
2         2  38 ContributorsLet’s Stay Together Lyrics[Intr...   
3         3  2 ContributorsUpgrade U (Beyoncé Experience Li...   
4         4  109 ContributorsTranslationsEspañol​hot girl b...   
...     ...                                                ...   
2205   1095  22 ContributorsChasin’ You Lyrics[Verse 1]\nWe...   
2206   1096  51 ContributorsDon’t Wanna Fall in Love Lyrics...   
2207   1097  46 ContributorsTranslationsPortuguêsItalianoNo...   
2208   1098  65 ContributorsStill into You Lyrics[Verse 1]\...   
2209   1099  35 ContributorsShe Got the Best of Me Lyrics[V...   

                     Song Name         Artist  Danceability  Energy  Key  \
0              I’m Good (Blue)   David Guetta         0.561   0.965    7   
1                    The Light     Juice WRLD         0

In [ ]:
#LANGCHAIN TIME
# here gonna take each lyric file and summarize it / vectorize it and store in pinecone
# initialize langchain / openAI

docs=[]
summary = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):  # check if the file is a .txt file
        file_path = os.path.join(directory, filename)
        # now you can do something with the file
        loader = TextLoader(file_path)
        doc = loader.load()
        docs.append(doc)
for i in tqdm(docs):
    summary.append(chain.run(i))

        

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-northeast1-gcp'
)

index_name="spotifysemantic"

for i in tqdm(docs):
    docsearch=Pinecone.from_documents(i, embeddings, index_name=index_name)





In [ ]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "songs for the best day of my life"
ans = docsearch.similarity_search(query)

for i in ans:
    print(i.metadata)